In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.5
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
seen_fr=[]
correct_count = 0
score_fr = 0
shuffled_sample = shuffled_words.sample(n=10)
for i in range(len(shuffled_sample)):
    if shuffled_sample['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample['Français'].iloc[i]} en hiragana ?")
        if is_similar_japanese_answer(ask_h, shuffled_sample['Hiragana'].iloc[i]):
            if shuffled_sample['Kanji'].iloc[i] != None:
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, shuffled_sample['Kanji'].iloc[i]):
                    seen_fr.append(shuffled_sample['Français'].iloc[i])
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {shuffled_sample['Kanji'].iloc[i]}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample)}')

if correct_count == len(shuffled_sample):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Comment dire sous-sol en hiragana ?ちした
Incorrect ! La bonne réponse est : ちか
Bonnes réponses : 0/10
Comment dire réparer en hiragana ?ｊ
Incorrect ! La bonne réponse est : しゅうりします
Bonnes réponses : 0/10
Comment dire chef de département en hiragana ?所長
Incorrect ! La bonne réponse est : ぶちょう
Bonnes réponses : 0/10
Comment dire avion en hiragana ?くうこう
Incorrect ! La bonne réponse est : ひこうき
Bonnes réponses : 0/10
Comment dire mari (de quelqu'un d'autre) en hiragana ?ひま
Incorrect ! La bonne réponse est : ごしゅじん
Bonnes réponses : 0/10
Comment dire temple bouddhiste en hiragana ?て
Incorrect ! La bonne réponse est : てら
Bonnes réponses : 0/10
Comment dire sous-vêtement en hiragana ?したぎ
Et en kanji ?下着
Correct !
Bonnes réponses : 1/10
Comment dire bureau en hiragana ?つくえ
Incorrect ! La bonne réponse est : じむしょ
Bonnes réponses : 1/10
Comment dire mairie en hiragana ?ｔ
Incorrect ! La bonne réponse est : しやくしょ
Bonnes réponses : 1/10
Comment dire se lever / réveiller en hiragana ?おきます
Et en kanji ?起

In [8]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample = shuffled_words.sample(n=10)
for i in range(len(shuffled_sample)):
    kanji = shuffled_sample['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample)}')

if correct_count == len(shuffled_sample):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Qu'est-ce que ce kanji : 名称?nom
Incorrecte ! La bonne réponse était : carte de visite
Bonnes réponses : 0/10
Qu'est-ce que ce kanji : 兄弟?frère et soeur
Correct ! C'était bien frères et soeurs
Bonnes réponses : 1/10
Qu'est-ce que ce kanji : 大学?université
Correct ! C'était bien université
Bonnes réponses : 2/10
Qu'est-ce que ce kanji : お土産?souvenirs
Correct ! C'était bien souvenir
Bonnes réponses : 3/10
Qu'est-ce que ce kanji : 飛行機?avion
Correct ! C'était bien avion
Bonnes réponses : 4/10
Qu'est-ce que ce kanji : 去年?année dernière
Correct ! C'était bien l'année dernière
Bonnes réponses : 5/10
Qu'est-ce que ce kanji : 金額?monnaie
Incorrecte ! La bonne réponse était : montant/somme d'argent
Bonnes réponses : 5/10
Qu'est-ce que ce kanji : 出張します?voyage d'affaire
Correct ! C'était bien faire un voyage d'affaires
Bonnes réponses : 6/10
Qu'est-ce que ce kanji : 試験?expérience
Incorrecte ! La bonne réponse était : examen
Bonnes réponses : 6/10
Qu'est-ce que ce kanji : 勉強します?étudier
Correct ! C'éta